## Climbing Shoe Website Parser
## Website: Bloc Shop


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners
import pyperclip
import json
#comment

Parse for max number of pages to iterate the parser through

In [9]:
url = "https://shop.blocshop.com/collections/all-climbing-shoes?page=3&section_id=template--15175211253894__main-collection"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find("div", class_="collection page-width page-width--full relative with-drawer")
product_elements = results.find_all("div", class_="card product-card product-card--card grid leading-none relative overflow-hidden")

for sale_element in product_elements:
    if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
        if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
            if len(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()) > 4:
                a = float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:]),
                print(a)

# print(results)

(151.0,)


Parsing through each webpage and converting it into a dataframe

In [10]:
df = pd.DataFrame(columns=["Shoe","Sale Price","Original Price","Discount","URL","Shipping Cost"])

for i in range(1,9):
    url = "https://shop.blocshop.com/collections/all-climbing-shoes?page={}&section_id=template--15175211253894__main-collection".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="collection page-width page-width--full relative with-drawer")
    product_elements = results.find_all("div", class_="card product-card product-card--card grid leading-none relative overflow-hidden")

    for sale_element in product_elements:
        if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
            if len(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()) > 8:
                new_row = {"Shoe" : sale_element.find("div", class_="product-card__top w-full").find("a").text.strip() 
                        + " " 
                        + sale_element.find("div", class_="product-card__details flex flex-col lg:flex-row items-baseline gap-2 w-full").find("a").text.strip(),
                "Sale Price" : float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:][0]),
                "Original Price" : float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),
                "Discount" : str(round((float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:])
                                        -float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:]))
                                    /float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),2)*100)+"%",
                "URL" : "shop.blocshop.com" + sale_element.find("a")["href"],
                "Shipping Cost" : 0
                }
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df],ignore_index=True)
            else:
                new_row = {"Shoe" : sale_element.find("div", class_="product-card__top w-full").find("a").text.strip() 
                        + " " 
                        + sale_element.find("div", class_="product-card__details flex flex-col lg:flex-row items-baseline gap-2 w-full").find("a").text.strip(),
                "Sale Price" : float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[1:]),
                "Original Price" : float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),
                "Discount" : str(round((float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:])
                                        -float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[1:]))
                                    /float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),2)*100)+"%",
                "URL" : "shop.blocshop.com" + sale_element.find("a")["href"],
                "Shipping Cost" : 0
                }
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df],ignore_index=True)


type_tiny = pyshorteners.Shortener() 
df["URL"] = df["URL"].apply(lambda x: type_tiny.tinyurl.short(x))
df["Post-Tax Price"] = (df["Sale Price"]+df["Shipping Cost"])*1.13
df["Post-Tax Price"] = df["Post-Tax Price"].apply(lambda x: round(x,2))

Output Table

In [11]:
#Output the table with values sorted
df.sort_values(by="Discount",ascending=False)

,Shoe,Sale Price,Original Price,Discount,URL,Shipping Cost,Post-Tax Price
5,Unparallel Sirius Lace LV,124.99,204.99,39.0%,https://tinyurl.com/23yoeq6u,0,141.24
6,Unparallel Sirius Lace,124.99,204.99,39.0%,https://tinyurl.com/23wkyykr,0,141.24
9,Unparallel TN Pro,124.99,204.99,39.0%,https://tinyurl.com/29scfjlw,0,141.24
7,Unparallel UP Duel,99.99,149.99,33.0%,https://tinyurl.com/2cgrf5xt,0,112.99
0,Black Diamond Momentum Women,80.47,114.95,30.0%,https://tinyurl.com/29zv7txa,0,90.93
14,Scarpa Mago,147.00,210.00,30.0%,https://tinyurl.com/28h3q8cu,0,166.11
10,Mad Rock Drone LV Black,128.00,178.00,28.000000000000004%,https://tinyurl.com/27wla2r5,0,144.64
2,La Sportiva Mantra 2023,149.96,199.95,25.0%,https://tinyurl.com/22y9web3,0,169.45
3,Evolv Zenist Women's 2023,157.50,210.00,25.0%,https://tinyurl.com/2bu3nxzj,0,177.97
4,Evolv Zenist 2023,157.50,210.00,25.0%,https://tinyurl.com/2cvdf2jc,0,177.97
